In [1]:
import os
import sys
from array import array
import ROOT as rt

import pyScripts.unfoldUtil as unfoldutil

Welcome to JupyROOT 6.14/04


In [2]:
class ISRAnalysis:
    
    def __init__(self, year_ = "2016", channel_= "electron", matrix_filekey_ = "matrix",
                 matrix_dirPath_ = "Detector_Dressed_DRp1_Fiducial", matrix_histName_ = "Detector_Dressed_DRp1"):
        
        # 디텍터 언폴딩 디렉토리 경로 & 매트릭스 이름 
        self.matrix_filekey = matrix_filekey_
        self.matrix_dirPath  = matrix_dirPath_
        self.matrix_histName = matrix_histName_
        
        self.channel = channel_
        self.year = year_
        self.nMassBins = None
       
        # Set data and Drell-Yan input histogram names
        dataHistPostfix = "Double"
        if self.channel == "electron":
            dataHistPostfix = dataHistPostfix + "EG"
            if self.year == "2018":
                dataHistPostfix = "EGamma"
        if self.channel == "muon":
            dataHistPostfix = dataHistPostfix + "Muon"
        self.dataHistName = "histo_"+dataHistPostfix+"nominal"
            
        self.dy10to50HistName = "DYJets10to50"
        MG_postfix = "_MG"
        if self.year != "2016":
            self.dy10to50HistName += MG_postfix
        
        # 아웃풋 디렉토리
        self.outDirPath = "output/"+self.year+"/new_"+self.channel+"/"
        # 인풋 히스토그램 텍스트파일
        self.inHistPathTxt = "inFiles/"+self.year+"/"+self.channel+"/fhist.txt"
    
        # Make output directory
        if not os.path.exists(self.outDirPath):
            os.makedirs(self.outDirPath)
    
        # Read text file including root file paths for unfolding
        self.filePaths = open(self.inHistPathTxt, 'r')
        self.inHistDic = {}
        
        for path in self.filePaths:
            modifiedPath = path.lstrip(' ').rstrip(' ').rstrip('\n')
            self.inHistDic[modifiedPath.split()[1]] = modifiedPath.split()[2]
        
        # 언폴딩 컨피규레이션
        self.bias = 1.0
        self.mode = 0 # 레귤라이제이션 모드
        
        # Create ISRUnfold object
        self.unfold = rt.ISRUnfold(self.channel, int(self.year), int(self.mode), False)
        self.unfold.setOutputBaseDir(self.outDirPath)
        self.unfold.setBias(self.bias)
        
        # Set response matrix
        self.unfold.setNomResMatrix("Pt", self.inHistDic[self.matrix_filekey], self.matrix_dirPath, self.matrix_histName)
        self.unfold.setNomResMatrix("Mass", self.inHistDic[self.matrix_filekey], self.matrix_dirPath, self.matrix_histName)
        
    def setSystematic():
        # Create systematic response matrix
        pass
    
    def setInputHist(self, useMCInput = False, useUnfoldOut = False, unfoldObj = None):
        
        inputHistName = self.dataHistName
        if useMCInput == True:
            if self.channel == "electron":
                inputHistName = "histo_DYJetsToEEnominal"
            else :
                inputHistName = "histo_DYJetsToMuMunominal"
        
        if useUnfoldOut == False:
            self.unfold.setUnfInput("Pt",   self.inHistDic['hist'], "Detector", inputHistName, False, "nominal", 0)
            self.unfold.setUnfInput("Mass", self.inHistDic['hist'], "Detector", inputHistName, False, "nominal", 0)
        else:
            self.unfold.setUnfInput(unfoldObj, "Pt", False, "", 0)
            self.unfold.setUnfInput(unfoldObj, "Mass", False, "", 0)
            
    def subFake(self):
        
        fakeList = {"DYJets": "DY", self.dy10to50HistName:"DY"}
        
        for fake in fakeList.items():
            self.unfold.subBkgs(self.inHistDic['matrix'], fake, False, "", 0, -1, "detector_level_DY_Fake")
        
    def setUnfoldBkgs(self, doSystematic = False , systName = "nominal", nthSys = 0, nTotSys = -1):
   
        bkgList = {}
        # 2016 데이터만 single top 샘플을 갖고 있다 
        if self.year == "2016" :
            bkgList = {"WJets_MG": "WJets", \
                       "WW_pythia": "EWK", "WZ_pythia": "EWK", "ZZ_pythia": "EWK", \
                       "DYJets10to50ToTauTau":"EWK", "DYJetsToTauTau":"EWK", \
                       "TTLL_powheg": "Top", "SingleTop_tW_top_Incl": "Top", "SingleTop_tW_antitop_Incl": "Top"}
        else :
            bkgList = {"WJets_MG": "WJets", \
                       "WW_pythia": "EWK", "WZ_pythia": "EWK", "ZZ_pythia": "EWK", \
                       "DYJets10to50ToTauTau":"EWK", "DYJetsToTauTau":"EWK", \
                       "TTLL_powheg": "Top"}
        
        for bkg in bkgList.items():
            self.unfold.subBkgs(self.inHistDic['hist'], bkg, doSystematic, systName, nTotSys, nthSys, "Detector")
            
    # Do unfold! 
    def doUnfold(self, doSystematic = False):
        self.unfold.doISRUnfold(doSystematic)
        
        # Set mean mass and pt values
        self.nMassBins = self.unfold.setMeanMass()
        self.unfold.setMeanPt()
    
    def getISRUnfold(self):
        
        return self.unfold
    
    # Get histograms
    def getPtVsMassTGraph(self, grTitle = "", isUnfolded = True):
        meanMass, meanPt = array('d'), array('d')
        meanMassStatErr, meanPtStatErr = array('d'), array('d')

        for ibin in range(self.nMassBins):
           
            if isUnfolded:
                meanMass.append(self.unfold.getUnfMeanMass(ibin))
                meanPt.append(self.unfold.getUnfMeanPt(ibin))
                meanMassStatErr.append(self.unfold.getUnfMeanMassError(ibin))
                meanPtStatErr.append(self.unfold.getUnfMeanPtError(ibin))
            else:
                meanMass.append(self.unfold.getDetMeanMass(ibin))
                meanPt.append(self.unfold.getDetMeanPt(ibin))
                meanMassStatErr.append(self.unfold.getDetMeanMassError(ibin))
                meanPtStatErr.append(self.unfold.getDetMeanPtError(ibin))
            
        gr = rt.TGraphErrors(self.nMassBins, meanMass, meanPt, meanMassStatErr, meanPtStatErr)
        gr.SetName(grTitle)
        return gr

In [3]:
import gc                                                                                                                                                                            
gc.collect() 

0

In [4]:
detUnfold_muon_2016 = ISRAnalysis("2016", "muon")

ISRUnfold set!
ISRUnfold::setNomResMatrix set response matrix...
0 th mass bin edge: 15
1 th mass bin edge: 40
2 th mass bin edge: 60
3 th mass bin edge: 81
4 th mass bin edge: 101
5 th mass bin edge: 200
6 th mass bin edge: 320
ISRUnfold::setNomResMatrix set response matrix...
ISRUnfold::setMassBindEdges massBinEdges already set.


Info in <TUnfoldV17::SetConstraint>: fConstraint=1
Warning in <TUnfoldV17::TUnfold>: 6 output bins do not depend on the input data  0-4 81
Info in <TUnfoldV17::TUnfold>: 80 input bins and 76 output bins
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #0
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #1 (Gen_Pt:mass[ufl]:pt[0,4])
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #2 (Gen_Pt:mass[ufl]:pt[4,8])
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #3 (Gen_Pt:mass[ufl]:pt[8,12])
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #4 (Gen_Pt:mass[ufl]:pt[12,18])
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #81
Info in <TUnfoldV17::SetConstraint>: fConstraint=1
Info in <TUnfoldV17::TUnfold>: underflow and overflow bin do not depend on the input data
Info in <TUnfoldV17::TUnfold>: 86 input bins and 86 output bins
Warning in <TUnfoldV17::TUnfold>: too few (ny=86) input bins for nx=86 output bins
Info in <TUnfoldDensityV17::TUn

In [5]:
detUnfold_muon_2017 = ISRAnalysis("2017", "muon")
detUnfold_muon_2018 = ISRAnalysis("2018", "muon")

ISRUnfold set!
ISRUnfold::setNomResMatrix set response matrix...
0 th mass bin edge: 15
1 th mass bin edge: 40
2 th mass bin edge: 60
3 th mass bin edge: 81
4 th mass bin edge: 101
5 th mass bin edge: 200
6 th mass bin edge: 320
ISRUnfold::setNomResMatrix set response matrix...
ISRUnfold::setMassBindEdges massBinEdges already set.
ISRUnfold set!
ISRUnfold::setNomResMatrix set response matrix...
0 th mass bin edge: 15
1 th mass bin edge: 40
2 th mass bin edge: 60
3 th mass bin edge: 81
4 th mass bin edge: 101
5 th mass bin edge: 200
6 th mass bin edge: 320
ISRUnfold::setNomResMatrix set response matrix...
ISRUnfold::setMassBindEdges massBinEdges already set.


Info in <TUnfoldV17::SetConstraint>: fConstraint=1
Warning in <TUnfoldV17::TUnfold>: 6 output bins do not depend on the input data  0-4 81
Info in <TUnfoldV17::TUnfold>: 80 input bins and 76 output bins
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #0
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #1 (Gen_Pt:mass[ufl]:pt[0,4])
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #2 (Gen_Pt:mass[ufl]:pt[4,8])
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #3 (Gen_Pt:mass[ufl]:pt[8,12])
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #4 (Gen_Pt:mass[ufl]:pt[12,18])
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #81
Info in <TUnfoldV17::SetConstraint>: fConstraint=1
Info in <TUnfoldV17::TUnfold>: underflow and overflow bin do not depend on the input data
Info in <TUnfoldV17::TUnfold>: 86 input bins and 86 output bins
Warning in <TUnfoldV17::TUnfold>: too few (ny=86) input bins for nx=86 output bins
Info in <TUnfoldDensityV17::TUn

In [6]:
fsrUnfold_muon_2016 = ISRAnalysis("2016", "muon", "fsr_matrix", 
                                      "Dressed_DRp1_Dressed_DR4PI_FullPhase", "Dressed_DRp1_Dressed_DR4PI")

fsrUnfold_muon_2017 = ISRAnalysis("2017", "muon", "fsr_matrix", 
                                      "Dressed_DRp1_Dressed_DR4PI_FullPhase", "Dressed_DRp1_Dressed_DR4PI")

fsrUnfold_muon_2018 = ISRAnalysis("2018", "muon", "fsr_matrix", 
                                      "Dressed_DRp1_Dressed_DR4PI_FullPhase", "Dressed_DRp1_Dressed_DR4PI")

ISRUnfold set!
ISRUnfold::setNomResMatrix set response matrix...
0 th mass bin edge: 15
1 th mass bin edge: 40
2 th mass bin edge: 60
3 th mass bin edge: 81
4 th mass bin edge: 101
5 th mass bin edge: 200
6 th mass bin edge: 320
ISRUnfold::setNomResMatrix set response matrix...
ISRUnfold::setMassBindEdges massBinEdges already set.
ISRUnfold set!
ISRUnfold::setNomResMatrix set response matrix...
0 th mass bin edge: 15
1 th mass bin edge: 40
2 th mass bin edge: 60
3 th mass bin edge: 81
4 th mass bin edge: 101
5 th mass bin edge: 200
6 th mass bin edge: 320
ISRUnfold::setNomResMatrix set response matrix...
ISRUnfold::setMassBindEdges massBinEdges already set.
ISRUnfold set!
ISRUnfold::setNomResMatrix set response matrix...
0 th mass bin edge: 15
1 th mass bin edge: 40
2 th mass bin edge: 60
3 th mass bin edge: 81
4 th mass bin edge: 101
5 th mass bin edge: 200
6 th mass bin edge: 320
ISRUnfold::setNomResMatrix set response matrix...
ISRUnfold::setMassBindEdges massBinEdges already set.


Info in <TUnfoldV17::SetConstraint>: fConstraint=1
Warning in <TUnfoldV17::TUnfold>: 6 output bins do not depend on the input data  0-4 81
Info in <TUnfoldV17::TUnfold>: 80 input bins and 76 output bins
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #0
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #1 (Gen_Pt:mass[ufl]:pt[0,4])
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #2 (Gen_Pt:mass[ufl]:pt[4,8])
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #3 (Gen_Pt:mass[ufl]:pt[8,12])
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #4 (Gen_Pt:mass[ufl]:pt[12,18])
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #81
Info in <TUnfoldV17::SetConstraint>: fConstraint=1
Info in <TUnfoldV17::TUnfold>: underflow and overflow bin do not depend on the input data
Info in <TUnfoldV17::TUnfold>: 86 input bins and 86 output bins
Warning in <TUnfoldV17::TUnfold>: too few (ny=86) input bins for nx=86 output bins
Info in <TUnfoldDensityV17::TUn

In [7]:
detUnfold_muon_2016.setInputHist()
detUnfold_muon_2016.setUnfoldBkgs()

detUnfold_muon_2017.setInputHist()
detUnfold_muon_2017.setUnfoldBkgs()

detUnfold_muon_2018.setInputHist()
detUnfold_muon_2018.setUnfoldBkgs()

Warning in <TUnfoldDensityV17::SetInput>: 4/80 input bins have zero error, and are ignored.
Warning in <TUnfoldDensityV17::SetInput>: 4/80 input bins have zero error, and are ignored.
Warning in <TUnfoldDensityV17::SetInput>: 4/80 input bins have zero error, and are ignored.


In [8]:
detUnfold_muon_2016.subFake()
detUnfold_muon_2017.subFake()
detUnfold_muon_2018.subFake()

In [9]:
detUnfold_muon_2016.doUnfold()
detUnfold_muon_2017.doUnfold()
detUnfold_muon_2018.doUnfold()

ISRUnfold::setMeanMass()   Save mean of dilepton...
ISRUnfold::setMeanPt()   Save mean of dilepton momentum...
ISRUnfold::setMeanMass()   Save mean of dilepton...
ISRUnfold::setMeanPt()   Save mean of dilepton momentum...
ISRUnfold::setMeanMass()   Save mean of dilepton...
ISRUnfold::setMeanPt()   Save mean of dilepton momentum...


Warning in <TUnfoldDensityV17::GetInputInverseEmatrix>: number of parameters 76 = input rank 76. Problem is ill posed
Warning in <TUnfoldDensityV17::GetInputInverseEmatrix>: number of parameters 86 = input rank 86. Problem is ill posed
Warning in <TUnfoldDensityV17::GetInputInverseEmatrix>: number of parameters 76 = input rank 76. Problem is ill posed
Warning in <TUnfoldDensityV17::GetInputInverseEmatrix>: number of parameters 86 = input rank 86. Problem is ill posed
Warning in <TUnfoldDensityV17::GetInputInverseEmatrix>: number of parameters 76 = input rank 76. Problem is ill posed
Warning in <TUnfoldDensityV17::GetInputInverseEmatrix>: number of parameters 86 = input rank 86. Problem is ill posed


In [10]:
gr_2016_muon_det_unfold =  detUnfold_muon_2016.getPtVsMassTGraph("2016MuonDetUnf")
gr_2017_muon_det_unfold =  detUnfold_muon_2017.getPtVsMassTGraph("2017MuonDefUnf")
gr_2018_muon_det_unfold =  detUnfold_muon_2018.getPtVsMassTGraph("2018MuonDefUnf")

gr_2016_muon_det =  detUnfold_muon_2016.getPtVsMassTGraph("2016MuonDet", False)
gr_2017_muon_det =  detUnfold_muon_2017.getPtVsMassTGraph("2017MuonDet", False)
gr_2018_muon_det =  detUnfold_muon_2018.getPtVsMassTGraph("2018MuonDet", False)

In [11]:
fsrUnfold_muon_2016.setInputHist(False, True, detUnfold_muon_2016.getISRUnfold())
fsrUnfold_muon_2016.doUnfold()

fsrUnfold_muon_2017.setInputHist(False, True, detUnfold_muon_2017.getISRUnfold())
fsrUnfold_muon_2017.doUnfold()

fsrUnfold_muon_2018.setInputHist(False, True, detUnfold_muon_2018.getISRUnfold())
fsrUnfold_muon_2018.doUnfold()

ISRUnfold::setMeanMass()   Save mean of dilepton...
ISRUnfold::setMeanPt()   Save mean of dilepton momentum...
ISRUnfold::setMeanMass()   Save mean of dilepton...
ISRUnfold::setMeanPt()   Save mean of dilepton momentum...
ISRUnfold::setMeanMass()   Save mean of dilepton...
ISRUnfold::setMeanPt()   Save mean of dilepton momentum...


Warning in <TUnfoldDensityV17::SetInput>: 4/80 input bins have zero error, and are ignored.
Warning in <TUnfoldDensityV17::GetInputInverseEmatrix>: number of parameters 76 = input rank 76. Problem is ill posed
Warning in <TUnfoldDensityV17::GetInputInverseEmatrix>: number of parameters 86 = input rank 86. Problem is ill posed
Warning in <TUnfoldDensityV17::SetInput>: 4/80 input bins have zero error, and are ignored.
Warning in <TUnfoldDensityV17::GetInputInverseEmatrix>: number of parameters 76 = input rank 76. Problem is ill posed
Warning in <TUnfoldDensityV17::GetInputInverseEmatrix>: number of parameters 86 = input rank 86. Problem is ill posed
Warning in <TUnfoldDensityV17::SetInput>: 4/80 input bins have zero error, and are ignored.
Warning in <TUnfoldDensityV17::GetInputInverseEmatrix>: number of parameters 76 = input rank 76. Problem is ill posed
Warning in <TUnfoldDensityV17::GetInputInverseEmatrix>: number of parameters 86 = input rank 86. Problem is ill posed


In [12]:
gr_2016_muon_fsr_unfold = fsrUnfold_muon_2016.getPtVsMassTGraph("2016MuonFSRUnf")
gr_2017_muon_fsr_unfold = fsrUnfold_muon_2017.getPtVsMassTGraph("2017MuonFSRUnf")
gr_2018_muon_fsr_unfold = fsrUnfold_muon_2018.getPtVsMassTGraph("2018MuonFSRUnf")

In [13]:
c_PtVsMass = rt.TCanvas("PtVsMass","PtVsMass", 1000, 600)
#pad = rt.TPad("pad", "pad", 0., 0., 1., 1.)

#pad.Draw()
#pad.SetGridx()
#pad.SetGridy()
#pad.SetLogx()

#c_PtVsMass.SetGridx()
#c_PtVsMass.SetGridy()
c_PtVsMass.SetLogx()

c_PtVsMass.SetBottomMargin(0.2)

In [18]:
import pyScripts.tdrStyle as tdrStyle
import pyScripts.CMS_lumi as CMS_lumi
tdrStyle.setTDRStyle()
rt.gStyle.SetOptFit(0)

In [19]:
#pad.cd()

gr_2016_muon_det_unfold.SetTitle("2016, 2017, 2018 muon")
gr_2016_muon_det_unfold.Draw("AP")
gr_2017_muon_det_unfold.Draw("P same")
gr_2018_muon_det_unfold.Draw("P same")
gr_2016_muon_det_unfold.SetMarkerStyle(20)
gr_2016_muon_det_unfold.SetMarkerSize(1)
gr_2017_muon_det_unfold.SetMarkerStyle(25)
gr_2017_muon_det_unfold.SetMarkerSize(1)
gr_2018_muon_det_unfold.SetMarkerStyle(26)
gr_2018_muon_det_unfold.SetMarkerSize(1)

gr_2016_muon_det.Draw("P SAME")
gr_2016_muon_det.SetMarkerStyle(20)
gr_2016_muon_det.SetMarkerSize(1)
gr_2016_muon_det.SetMarkerColor(rt.kBlue)
gr_2016_muon_det.SetLineColor(rt.kBlue)
gr_2017_muon_det.Draw("P SAME")
gr_2017_muon_det.SetMarkerStyle(25)
gr_2017_muon_det.SetMarkerSize(1)
gr_2017_muon_det.SetMarkerColor(rt.kBlue)
gr_2017_muon_det.SetLineColor(rt.kBlue)
gr_2018_muon_det.Draw("P SAME")
gr_2018_muon_det.SetMarkerStyle(26)
gr_2018_muon_det.SetMarkerSize(1)
gr_2018_muon_det.SetMarkerColor(rt.kBlue)
gr_2018_muon_det.SetLineColor(rt.kBlue)

gr_2016_muon_fsr_unfold.Draw("P same")
gr_2016_muon_fsr_unfold.SetMarkerStyle(20)
gr_2016_muon_fsr_unfold.SetMarkerColor(rt.kRed)
gr_2016_muon_fsr_unfold.SetLineColor(rt.kRed)
gr_2016_muon_fsr_unfold.SetMarkerSize(1)
gr_2017_muon_fsr_unfold.Draw("P same")
gr_2017_muon_fsr_unfold.SetMarkerStyle(25)
gr_2017_muon_fsr_unfold.SetMarkerColor(rt.kRed)
gr_2017_muon_fsr_unfold.SetLineColor(rt.kRed)
gr_2017_muon_fsr_unfold.SetMarkerSize(1)
gr_2018_muon_fsr_unfold.Draw("P same")
gr_2018_muon_fsr_unfold.SetMarkerStyle(26)
gr_2018_muon_fsr_unfold.SetMarkerColor(rt.kRed)
gr_2018_muon_fsr_unfold.SetLineColor(rt.kRed)
gr_2018_muon_fsr_unfold.SetMarkerSize(1)

gr_2016_muon_det_unfold.GetYaxis().SetRangeUser(10., 30.)
gr_2016_muon_det_unfold.GetXaxis().SetLimits(35., 350.)
gr_2016_muon_det_unfold.GetYaxis().SetTitle("Dilepton p_{T} [GeV]")
gr_2016_muon_det_unfold.GetXaxis().SetTitle("Dilepton Mass [GeV]")
gr_2016_muon_det_unfold.GetXaxis().SetTitleOffset(1.2)
gr_2016_muon_det_unfold.GetXaxis().SetMoreLogLabels(True)

CMS_lumi.extraText = "Work in progress"
CMS_lumi.CMS_lumi(c_PtVsMass, 0, 11)

legend = rt.TLegend(0.7, 0.3, 0.9, 0.5)
legend.SetBorderSize(1);
legend.AddEntry(gr_2016_muon_det_unfold, "Detector data 2016", "ple")
legend.Draw()

# Linear fit
fitLinear = rt.TF1("f_lepton", "[0]+[1]*log(x)", 10., 400.);
fitLinear.SetLineStyle(2)
fitLinear.SetLineColor(rt.kBlue)
fitLinear.SetLineWidth(1)

gr_2016_muon_fsr_unfold.Fit(fitLinear, "R0")
c_PtVsMass.Draw()


 FCN=244.615 FROM MIGRAD    STATUS=CONVERGED      57 CALLS          58 TOTAL
                     EDM=5.70807e-13    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0          -1.70454e+01   1.42704e-01   3.56719e-05   2.24525e-05
   2  p1           7.82037e+00   3.17834e-02   7.94494e-06   1.34211e-04


In [20]:
%jsroot on

In [17]:
c_PtVsMass.SaveAs("Muon.pdf")

Info in <TCanvas::Print>: pdf file Muon.pdf has been created
